# Image Recommendation System via Collaborative Filtering

# ***Please read the instructions very carefully***

1.   Assignment must be implemented in Python 3 only.
2.   You are allowed to use libraries for data preprocessing (numpy, pandas, nltk etc) and for evaluation metrics, data visualization (matplotlib etc.).
3.   You will be evaluated not just on the overall performance of the model and also on the experimentation with hyper parameters, data prepossessing techniques etc.
4.   ⚠️ The Assignment will be evaluated automatically. Please adhere to taking proper inputs from `config.csv` file. You can change your `config.csv` file to experiment with your code. But at the end, make sure that your outputs are corresponding to input values in `config.csv`
5.   Strict plagiarism checking will be done. An F will be awarded for plagiarism.

## About the Dataset
Behance is a community art website where users showcase and discover creative work. Each user is able to “appreciate” (equivalent to a “like” on Instagram or a “react” on Facebook) an image, indicating that they like the image. It is in the website’s best interests to show users pictures that they would like, to keep them engaged for longer. For this question, given a set of pictures that a user has already appreciated, you have to show them a new picture that they would like based on what similar users appreciated.


<br><br>
**The dataset has information of 1 million appreciates of 63,497 users on 178,788 items. The file Behance appreciate 1M has a triplet in each line in the form of (user id, item id, unix timestamp).**

**Task: Take the inputs from the config.csv file and output the recommendations for a particular person**
- Collaborative Filtering is a way to predict items to the user based on the the
user’s history and the history of similar users. The similarity between users can be quantified by the number of images that both the users appreciated.
- The images appreciated by a similar user would be the most suitable images to show a user. Since we can find the similarity between any two users, we would be able to find the “nearest” neighbours of any user, allowing us to use a KNN-based algorithm to recommend new images to a user.
- Since people do not like seeing pictures that they have seen already. Make sure that you do not recommend pictures that a user has appreciated already.
- Output the final response will be saved in the file named ```config['output_file']```.


**Output file format:**
Populate the output file with images that the user has not seen of the k most
similar users, in descending order of their similarity. Each line in the output
file should be a duplet in the form of (item id, user id), where the user id is the
id of the kth similar user. The order of the images corresponding to the same
similar user would not matter. The output file would look something like this:
```
item_id_1_of_1st_similar_user 1st_most_similar_user_id
item_id_2_of_1st_similar_user 1st_most_similar_user_id
item_id_3_of_1st_similar_user 1st_most_similar_user_id
...
item_id_1_of_2nd_similar_user 2nd_most_similar_user_id
item_id_2_of_2nd_similar_user 2nd_most_similar_user_id
item_id_3_of_2nd_similar_user 2nd_most_similar_user_id
...
item_id_1_of_kth_similar_user kth_most_similar_user_id
item_id_2_of_kth_similar_user kth_most_similar_user_id
item_id_3_of_kth_similar_user kth_most_similar_user_id
```

The dataset was extracted using Behance’s API as a part of the paper
“Vista: A visually, socially, and temporally-aware model for artistic
recommendation, RecSys, 2016”. Check out this [Google Drive folder](https://drive.google.com/drive/folders/0B9Ck8jw-TZUEc3NlMjVXdDlPU1k?resourcekey=0-6_8ykn0o4fLc5fuTEm91xA) for
more information about the dataset.


Have fun! The users are waiting to see new pictures!

### Import necessary libraries

In [59]:
import numpy as np
import pandas as pd

In [60]:
# Config Generation Sample Code.
# ⚠️ Only for experimentation on your side.
# ⚠️ Should be commented during submission.

# df = pd.DataFrame(data=[{'id':276633,
#                   'k':5,
#                   'dataset_file':'./Behance_appreciate_1M',
#                   'output_file':'./output.txt'}])
# df.to_csv('config.csv')

In [61]:
config = pd.read_csv('config.csv').iloc[0]

In [62]:
config

Unnamed: 0                            0
id                               276633
k                                     5
dataset_file    ./Behance_appreciate_1M
output_file                ./output.txt
Name: 0, dtype: object

In [63]:
user = config['id']
k_value = config['k']

### Read the Data

In [64]:
with open(config['dataset_file'], 'r') as inFile:
    appreciate_data = inFile.readlines()

### Initialize a dictionary to store the item_ids that a user likes

### Go through each line of the input file and construct the user_likes dictionary

In [65]:
user_likes = dict()
user_similarity = dict()

In [66]:
for line in appreciate_data:
    line = line.strip()
    
    user_id = int(line.split()[0])
    item_id = int(line.split()[1])

    if user_id not in user_likes:
        user_likes[user_id] = list()
        user_similarity[user_id] = 0 
    user_likes[user_id].append(item_id)

### Use KNN after Collaborative Filtering to find nearest neighbors

In [67]:
for liked_item in user_likes[user]:
    for a_user in user_likes:
        if liked_item in user_likes[a_user]:
            user_similarity[a_user] += 1
print(user_similarity)

{276633: 203, 1238354: 29, 165550: 2, 2173258: 3, 2459267: 0, 3192280: 1, 82097: 0, 119968: 1, 16235: 4, 666917: 38, 27140: 2, 3735089: 0, 872240: 0, 405186: 0, 2421008: 0, 2616071: 0, 1340996: 0, 1574720: 3, 2680771: 0, 2373710: 1, 180376: 0, 633007: 0, 173110: 0, 1729053: 0, 473711: 0, 2293332: 0, 2842319: 0, 1465758: 15, 1651180: 0, 931515: 1, 129327: 3, 1972439: 0, 1823010: 1, 805401: 0, 2629364: 0, 2841523: 3, 1361763: 2, 3907125: 0, 298504: 0, 1212606: 1, 2294038: 0, 1865995: 1, 2121302: 0, 1875152: 0, 2536306: 1, 2513234: 1, 3280048: 2, 148262: 0, 1916012: 1, 2414370: 2, 1910538: 0, 2863549: 6, 2331756: 1, 2299123: 0, 3297927: 0, 3536928: 0, 71523: 0, 3247141: 1, 2822775: 0, 3659128: 10, 1432503: 2, 1143296: 0, 1038564: 0, 2678811: 5, 1785543: 1, 1093108: 8, 723504: 0, 1347443: 0, 718858: 0, 2328172: 0, 3562234: 1, 275993: 0, 2784505: 0, 2478593: 1, 1945282: 0, 2960573: 0, 100318: 1, 3212195: 3, 1265647: 1, 2288055: 0, 906198: 2, 1937735: 0, 2770373: 0, 1596945: 0, 2095827: 0, 4

In [68]:
def neighbors(user,k_value):
    k_n_n = []
    for i in range(k_value):
        max_sim = 0
        max_sim_user = 0
        for a_user in user_similarity:
            if a_user in k_n_n:
                continue
            elif max_sim <= user_similarity[a_user]:
                max_sim_user = a_user
                max_sim = user_similarity[a_user]
        k_n_n.append(max_sim_user)
    return k_n_n

In [69]:
knn = neighbors(user, 7)

In [70]:
knn

[276633, 3628520, 2492899, 666917, 3313136, 2585131, 3862918]

In [57]:
for i in range(7):
    print(user_similarity[knn[i]])

203
41
38
38
37
35
29


### Open the output file to write all the lines to the file

In [72]:
outFile = open(config['output_file'], 'w')
knn = neighbors(user,k_value)
for user_id in knn:
    for item_id in user_likes[user_id]:
        if item_id in user_likes[user]:
            continue
        else:
            outFile.write(str(item_id) + ' ' + str(user_id) + '\n')

outFile.close()